# Third Notebook

In this notebook, the JSON files created from the reduced dataset will be integrated for training. A pretrained Faster R-CNN model will be used due to its accuracy and resource efficiency. Additionally, model tracking in MLflow will be initiated.

## Objectives

1. Load the reduced dataset generated in `data/processed/`.
2. Create a DataLoader compatible with detection models (torchvision).
3. Build a pretrained model (Faster R-CNN) and adapt the head to 3 classes.
4. Train on CPU with an efficient configuration.
5. Log to MLflow:
   - parameters
   - metrics
   - artifacts (weights, labelmap, config)
6. Save a local checkpoint for inference and for incremental retraining.

In [ ]:
"""
- Imports the necessary libraries for training.
"""

import os
import json
import time
from pathlib import Path
from datetime import datetime
from typing import Dict, List

import random
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image

from tqdm import tqdm

import mlflow

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F


1. Loading .json files and validating the project structure

In [ ]:
"""
In this section, project_config.json and labelmap.json created in Notebook 01 are loaded.
It also defines absolute paths without depending on the notebook’s current working directory.
"""


def find_project_root(start: Path, max_up: int = 8) -> Path:
    cur = start.resolve()
    for _ in range(max_up):
        if (cur / "data" / "processed" / "project_config.json").exists():
            return cur
        cur = cur.parent
    raise FileNotFoundError("No se encontró data/processed/project_config.json. Ejecuta Notebook 01.")

PROJECT_ROOT = find_project_root(Path.cwd())
PROCESSED_DIR = (PROJECT_ROOT / "data" / "processed").resolve()

PROJECT_CONFIG_PATH = (PROCESSED_DIR / "project_config.json").resolve()
LABELMAP_PATH = (PROCESSED_DIR / "labelmap.json").resolve()

with open(PROJECT_CONFIG_PATH, "r", encoding="utf-8") as f:
    project_config = json.load(f)

with open(LABELMAP_PATH, "r", encoding="utf-8") as f:
    labelmap = json.load(f)

COCO_ROOT = Path(project_config["coco_root"])
TRAIN_IMG_DIR = Path(project_config["train_dir"])
VAL_IMG_DIR = Path(project_config["val_dir"])

TARGET_CLASSES = project_config["target_classes"]
target_cat_ids = labelmap["target_category_ids"]

TRAIN_JSON = (PROCESSED_DIR / "coco_person_car_airplane_train.json").resolve()
VAL_JSON = (PROCESSED_DIR / "coco_person_car_airplane_val.json").resolve()

MODELS_DIR = (PROJECT_ROOT / "models" / "local_checkpoints").resolve()
MODELS_DIR.mkdir(parents=True, exist_ok=True)

print("Mismo contenido visto anteriormente, solo para confirmar las carpetas ")
print("PROJECT_ROOT:", PROJECT_ROOT)
print("TRAIN_JSON:", TRAIN_JSON)
print("VAL_JSON  :", VAL_JSON)
print("TARGET_CLASSES:", TARGET_CLASSES)
print("target_cat_ids:", target_cat_ids)
print("MODELS_DIR:", MODELS_DIR)


PROJECT_ROOT: C:\Users\Johnny\Desktop\IA-final
TRAIN_JSON: C:\Users\Johnny\Desktop\IA-final\data\processed\coco_person_car_airplane_train.json
VAL_JSON  : C:\Users\Johnny\Desktop\IA-final\data\processed\coco_person_car_airplane_val.json
TARGET_CLASSES: ['person', 'car', 'airplane']
target_cat_ids: [1, 3, 5]
MODELS_DIR: C:\Users\Johnny\Desktop\IA-final\models\local_checkpoints


2. Limited training configuration

In [ ]:
"""
In this section, CPU-based training is configured, since a powerful GPU is not available on the machine used for training.
If you have access to a high-performance GPU, you can adjust these parameters to speed up the training process.
Additionally, threads and random seeds are set to ensure reproducibility and prevent random image selection.
Conservative hyperparameters suitable for CPU training are also defined.
"""

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = torch.device("cpu")

CPU_NUM_THREADS = max(1, (os.cpu_count() or 2) - 1)
torch.set_num_threads(CPU_NUM_THREADS)

CONFIG = {
    "seed": SEED,
    "device": str(DEVICE),
    "cpu_num_threads": CPU_NUM_THREADS,
    "batch_size": 2,
    "num_workers": 0,
    "learning_rate": 1e-4,
    "weight_decay": 1e-4,
    "epochs": 2,
    "score_threshold": 0.5,
    "model_name": "fasterrcnn_resnet50_fpn",
    "pretrained_weights": "DEFAULT",
}

print("CONFIG:")
for k, v in CONFIG.items():
    print(f"{k}: {v}")


CONFIG:
seed: 42
device: cpu
cpu_num_threads: 11
batch_size: 2
num_workers: 0
learning_rate: 0.0001
weight_decay: 0.0001
epochs: 2
score_threshold: 0.5
model_name: fasterrcnn_resnet50_fpn
pretrained_weights: DEFAULT


In [ ]:
"""
Here, the main goal is to verify that TRAIN_JSON and VAL_JSON exist.
It also uses the previously defined limits on the number of images they contain.
If any abnormal situation occurs, execution is halted if the training set exceeds the established limit.
"""

def assert_file(path: Path, label: str) -> None:
    if not path.exists():
        raise FileNotFoundError(f"Archivo requerido no encontrado: {label}: {path}")

assert_file(TRAIN_JSON, "TRAIN_JSON reducido (Notebook 02)")
assert_file(VAL_JSON, "VAL_JSON reducido (Notebook 02)")

with open(TRAIN_JSON, "r", encoding="utf-8") as f:
    train_coco = json.load(f)

with open(VAL_JSON, "r", encoding="utf-8") as f:
    val_coco = json.load(f)

train_images_n = len(train_coco["images"])
val_images_n = len(val_coco["images"])

print("TRAIN images:", train_images_n)
print("VAL images  :", val_images_n)

MAX_ALLOWED_TRAIN_IMAGES = 10000  # seguridad dura
if train_images_n > MAX_ALLOWED_TRAIN_IMAGES:
    raise RuntimeError(
        f"TRAIN_JSON tiene {train_images_n} imágenes, supera el límite {MAX_ALLOWED_TRAIN_IMAGES}. "
        "Regenera el JSON en Notebook 02 con recorte."
    )


TRAIN images: 3300
VAL images  : 500


In [ ]:
"""
In this section, a custom Dataset is defined using the dataset previously reduced in the second notebook, designed to work with COCO-format data and specifically adapted for torchvision object detection models.
The main objective is to load the images, process their annotations, and return the data in the exact format required by the model for training.
"""


class CocoReducedDetectionDataset(Dataset):
    def __init__(self, images_dir: Path, coco_json: dict):
        self.images_dir = images_dir
        self.coco = coco_json

        self.images = self.coco["images"]
        self.annotations = self.coco["annotations"]

        self.img_id_to_anns = {}
        for ann in self.annotations:
            self.img_id_to_anns.setdefault(ann["image_id"], []).append(ann)

        self.id_to_image = {img["id"]: img for img in self.images}
        self.image_ids = list(self.id_to_image.keys())

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx: int):
        img_id = self.image_ids[idx]
        img_meta = self.id_to_image[img_id]
        img_path = self.images_dir / img_meta["file_name"]

        img = Image.open(img_path).convert("RGB")
        img_tensor = F.to_tensor(img)

        anns = self.img_id_to_anns.get(img_id, [])

        boxes = []
        labels = []
        areas = []
        iscrowd = []

        for a in anns:
            x, y, w, h = a["bbox"]
            boxes.append([x, y, x + w, y + h])
            labels.append(a["category_id"])
            areas.append(a.get("area", w * h))
            iscrowd.append(a.get("iscrowd", 0))

        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64),
            "image_id": torch.tensor([img_id], dtype=torch.int64),
            "area": torch.tensor(areas, dtype=torch.float32),
            "iscrowd": torch.tensor(iscrowd, dtype=torch.int64),
        }

        return img_tensor, target


def collate_fn(batch):
    images, targets = zip(*batch)
    return list(images), list(targets)


In [ ]:
"""
In this section, the training and validation datasets are created, and then the DataLoaders that will be used during model training are built.
It also ensures that train_loader and val_loader exist before starting the training process.
"""

train_ds = CocoReducedDetectionDataset(TRAIN_IMG_DIR, train_coco)
val_ds = CocoReducedDetectionDataset(VAL_IMG_DIR, val_coco)

train_loader = DataLoader(
    train_ds,
    batch_size=CONFIG["batch_size"],
    shuffle=True,
    num_workers=CONFIG["num_workers"],
    collate_fn=collate_fn
)

val_loader = DataLoader(
    val_ds,
    batch_size=1,
    shuffle=False,
    num_workers=CONFIG["num_workers"],
    collate_fn=collate_fn
)

print("Train samples:", len(train_ds))
print("Val samples  :", len(val_ds))
print("Train iters per epoch:", len(train_loader))


Train samples: 3300
Val samples  : 500
Train iters per epoch: 1650


2. Model construction

In [ ]:
"""
In this section, the Faster R-CNN detection model is built using pretrained weights,
and its architecture is adjusted to work exclusively with the project’s target classes.
Since COCO category identifiers are not consecutive and do not start from 1,
the original category_id values are remapped to internal labels ranging from 1..K,
with 0 reserved for the background class, as required by the model.
Additionally, the classification head of the model is redefined so that the number of outputs matches the total number of target classes plus the background class.
"""

target_cat_ids_int = [int(x) for x in target_cat_ids]
coco_to_internal = {cid: i + 1 for i, cid in enumerate(target_cat_ids_int)}
internal_to_name = {i + 1: name for i, name in enumerate(TARGET_CLASSES)}

NUM_CLASSES = len(TARGET_CLASSES) + 1

def remap_targets(targets: List[Dict]) -> List[Dict]:
    out = []
    for t in targets:
        new_t = t.copy()
        labels = t["labels"].clone()
        for i in range(labels.shape[0]):
            labels[i] = coco_to_internal[int(labels[i].item())]
        new_t["labels"] = labels
        out.append(new_t)
    return out

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=CONFIG["pretrained_weights"])
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES)

model.to(DEVICE)
model.train()

print("NUM_CLASSES:", NUM_CLASSES)
print("coco_to_internal:", coco_to_internal)


NUM_CLASSES: 4
coco_to_internal: {1: 1, 3: 2, 5: 3}


In [ ]:
"""
In this section, the AdamW optimizer is defined to update the model’s weights during training, as it provides good stability and helps control overfitting through weight_decay.
Additionally, the epoch-level training (train_one_epoch) and evaluation (evaluate_loss) functions are implemented, where the average loss per epoch is computed.
This is particularly useful for monitoring CPU-based training and for comparing performance between training and validation phases.
"""

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=CONFIG["learning_rate"],
    weight_decay=CONFIG["weight_decay"]
)

def train_one_epoch(model, data_loader, optimizer, epoch: int) -> float:
    model.train()
    total_loss = 0.0
    n = 0

    for images, targets in tqdm(data_loader, desc=f"train e{epoch}", leave=False):
        images = [img.to(DEVICE) for img in images]
        targets = remap_targets(targets)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        total_loss += float(losses.item())
        n += 1

    return total_loss / max(1, n)

@torch.no_grad()
def evaluate_loss(model, data_loader) -> float:
    model.eval()
    total_loss = 0.0
    n = 0

    for images, targets in tqdm(data_loader, desc="val", leave=False):
        images = [img.to(DEVICE) for img in images]
        targets = remap_targets(targets)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        total_loss += float(losses.item())
        n += 1

    model.train()
    return total_loss / max(1, n)


3. Uso del entrenamiento con dataset reducido, modelo preentrenado y optimizador

In [ ]:
"""
This cell:
- Configures MLflow with SQLite (mlflow.db) to avoid FileStore deprecation issues.
- Logs parameters, metrics, and artifacts (mappings + con
"""
# -----------------------------
# MLflow: SQLite backend
# -----------------------------
MLFLOW_DB = (PROJECT_ROOT / "mlflow.db").resolve()
mlflow.set_tracking_uri(f"sqlite:///{MLFLOW_DB.as_posix()}")

EXPERIMENT_NAME = "object_detection_coco_cpu"
mlflow.set_experiment(EXPERIMENT_NAME)

run_name = f"base_train_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

best_val_loss = float("inf")
best_ckpt_path = None


@torch.no_grad()
def evaluate_loss(model, data_loader) -> float:
    """
    Esta función:
    - Calcula pérdida en validación sin gradientes.
    - En modelos de detección torchvision, el diccionario de pérdidas se obtiene en modo train.
    - Por eso forzamos model.train() temporalmente y luego restauramos el estado previo.
    """
    was_training = model.training
    model.train()

    total_loss = 0.0
    n = 0

    for images, targets in tqdm(data_loader, desc="val", leave=False):
        images = [img.to(DEVICE) for img in images]
        targets = remap_targets(targets)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)  # dict de pérdidas
        losses = sum(loss for loss in loss_dict.values())

        total_loss += float(losses.item())
        n += 1

    if not was_training:
        model.eval()

    return total_loss / max(1, n)


with mlflow.start_run(run_name=run_name):
    # -----------------------------
    # Params + tags
    # -----------------------------
    mlflow.log_params(CONFIG)
    mlflow.set_tag("stage", "base")
    mlflow.set_tag("train_type", "cpu")
    mlflow.set_tag("classes", ",".join(TARGET_CLASSES))
    mlflow.set_tag("model_arch", CONFIG["model_name"])

    # -----------------------------
    # Artefactos de configuración/mapping
    # -----------------------------
    mapping_artifact = {
        "target_classes": TARGET_CLASSES,
        "target_category_ids": target_cat_ids_int,
        "coco_to_internal": coco_to_internal,
        "internal_to_name": internal_to_name,
        "train_json": str(TRAIN_JSON.as_posix()),
        "val_json": str(VAL_JSON.as_posix()),
    }

    mapping_path = MODELS_DIR / f"mapping_{run_name}.json"
    with open(mapping_path, "w", encoding="utf-8") as f:
        json.dump(mapping_artifact, f, indent=2, ensure_ascii=False)

    mlflow.log_artifact(str(mapping_path), artifact_path="artifacts")
    mlflow.log_artifact(str(PROJECT_CONFIG_PATH), artifact_path="artifacts")
    mlflow.log_artifact(str(LABELMAP_PATH), artifact_path="artifacts")

    # -----------------------------
    # Entrenamiento por épocas
    # -----------------------------
    for epoch in range(1, CONFIG["epochs"] + 1):
        t0 = time.time()

        # TRAIN
        train_loss = train_one_epoch(model, train_loader, optimizer, epoch)

        # CHECKPOINT por época (SIEMPRE, antes de validar)
        epoch_ckpt_path = MODELS_DIR / f"epoch_{epoch}_frcnn_cpu_{run_name}.pt"
        epoch_ckpt = {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "config": CONFIG,
            "target_classes": TARGET_CLASSES,
            "target_category_ids": target_cat_ids_int,
            "coco_to_internal": coco_to_internal,
            "internal_to_name": internal_to_name,
        }
        torch.save(epoch_ckpt, epoch_ckpt_path)
        mlflow.log_artifact(str(epoch_ckpt_path), artifact_path="checkpoints")

        # VALIDATION (loss)
        val_loss = evaluate_loss(model, val_loader)

        epoch_time = time.time() - t0

        # LOG metrics
        mlflow.log_metric("train_loss", train_loss, step=epoch)
        mlflow.log_metric("val_loss", val_loss, step=epoch)
        mlflow.log_metric("epoch_time_sec", epoch_time, step=epoch)

        print(
            f"Epoch {epoch}/{CONFIG['epochs']} | "
            f"train_loss={train_loss:.4f} | val_loss={val_loss:.4f} | time={epoch_time:.1f}s"
        )

        # BEST model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_ckpt_path = MODELS_DIR / f"best_frcnn_cpu_{run_name}.pt"

            best_ckpt = epoch_ckpt.copy()
            best_ckpt["best_val_loss"] = best_val_loss

            torch.save(best_ckpt, best_ckpt_path)
            mlflow.log_artifact(str(best_ckpt_path), artifact_path="checkpoints")

    mlflow.log_metric("best_val_loss", best_val_loss)

print("Entrenamiento finalizado")
print("best_val_loss:", best_val_loss)
print("best_ckpt_path:", best_ckpt_path)


2026/02/01 08:34:46 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/01 08:34:46 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/01 08:34:46 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/01 08:34:46 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/01 08:34:46 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/01 08:34:46 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/01 08:34:48 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/01 08:34:48 INFO mlflow.store.db.utils: Updating database tables
2026/02/01 08:34:48 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/01 08:34:48 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/01 08:34:48 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/01 08:34:48 INFO alembic.runtime

Epoch 1/2 | train_loss=0.4747 | val_loss=0.4836 | time=21692.4s


Epoch 2/2 | train_loss=0.4070 | val_loss=0.4406 | time=20395.6s
Entrenamiento finalizado
best_val_loss: 0.44062818501889706
best_ckpt_path: C:\Users\Johnny\Desktop\IA-final\models\local_checkpoints\best_frcnn_cpu_base_train_20260201_083448.pt


In [ ]:
"""
This section is mainly used to ensure model reliability for
"""

@torch.no_grad()
def load_model_from_ckpt(ckpt_path: Path):
    ckpt = torch.load(ckpt_path, map_location="cpu")

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=None)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    num_classes = len(ckpt["target_classes"]) + 1
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    model.load_state_dict(ckpt["model_state_dict"])
    model.eval()
    return model, ckpt

@torch.no_grad()
def predict_top3(model, image_path: Path, internal_to_name: Dict[int, str], score_thr: float = 0.5):
    img = Image.open(image_path).convert("RGB")
    img_t = F.to_tensor(img)

    outputs = model([img_t])[0]
    boxes = outputs["boxes"].cpu()
    scores = outputs["scores"].cpu()
    labels = outputs["labels"].cpu()

    keep = scores >= score_thr
    boxes = boxes[keep]
    scores = scores[keep]
    labels = labels[keep]

    if len(scores) == 0:
        return []

    topk = min(3, len(scores))
    idx = torch.argsort(scores, descending=True)[:topk]

    results = []
    for i in idx:
        lbl = int(labels[i].item())
        results.append({
            "name": internal_to_name.get(lbl, "unknown"),
            "score": float(scores[i].item()),
            "box_xyxy": boxes[i].tolist(),
        })
    return results

if best_ckpt_path is None:
    raise FileNotFoundError("No hay best_ckpt_path. Revisa el entrenamiento.")

infer_model, ckpt = load_model_from_ckpt(best_ckpt_path)

sample_img_meta = val_coco["images"][0]
sample_path = VAL_IMG_DIR / sample_img_meta["file_name"]

preds = predict_top3(infer_model, sample_path, ckpt["internal_to_name"], score_thr=CONFIG["score_threshold"])

print("Imagen:", sample_path)
if len(preds) == 0:
    print("no se ha encontrado")
else:
    for p in preds:
        print(p)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Johnny/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:03<00:00, 31.1MB/s]


Imagen: C:\Users\Johnny\Desktop\IA-final\data\archive\coco2017\val2017\000000397133.jpg
{'name': 'person', 'score': 0.9935327768325806, 'box_xyxy': [385.1822204589844, 76.41061401367188, 497.1370544433594, 343.8106994628906]}
{'name': 'person', 'score': 0.9191341400146484, 'box_xyxy': [352.98193359375, 97.65465545654297, 404.5033264160156, 238.57272338867188]}
{'name': 'person', 'score': 0.6582873463630676, 'box_xyxy': [199.18922424316406, 9.169189453125, 283.14971923828125, 101.2300796508789]}
